# 读取数据集

In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('./data/train_set.csv')#训练集

test = pd.read_csv('./data/test_set.csv')#测试集

In [3]:
train.head()

id                                            article  \
0   0  7368 1252069 365865 755561 1044285 129532 1053...   
1   1  581131 165432 7368 957317 1197553 570900 33659...   
2   2  7368 87936 40494 490286 856005 641588 145611 1...   
3   3  299237 760651 299237 887082 159592 556634 7489...   
4   4  7368 7368 7368 865510 7368 396966 995243 37685...   

                                            word_seg  class  
0  816903 597526 520477 1179558 1033823 758724 63...     14  
1  90540 816903 441039 816903 569138 816903 10343...      3  
2  816903 1012629 957974 1033823 328210 947200 65...     12  
3  563568 1239563 680125 780219 782805 1033823 19...     13  
4  816903 816903 816903 139132 816903 312320 1103...     12

# TFIDF构建文本特征

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
word_vec = TfidfVectorizer(analyzer='word',
            ngram_range=(1,2),#(1,3)
            min_df=3,  # 4  5
            max_df=0.9, # 0.95 1.0 
            use_idf=True,
            smooth_idf=True, 
            sublinear_tf=True)

In [6]:
train_term_doc = word_vec.fit_transform(train['word_seg'])  #训练集

In [7]:
test_term_doc = word_vec.transform(test['word_seg'])  #预测测试集

In [8]:
train_term_doc#sparse matrix 102277x2820641  np.array 稀疏矩阵

<102277x2820641 sparse matrix of type '<class 'numpy.float64'>'
	with 75739932 stored elements in Compressed Sparse Row format>

In [9]:
test_term_doc  #102277x2820641 稀疏矩阵可以直接在逻辑回归里面调用

<102277x2820641 sparse matrix of type '<class 'numpy.float64'>'
	with 73582169 stored elements in Compressed Sparse Row format>

# 逻辑回归构建模型

In [14]:
[1,2,3,4,5,6,7,19]  -> [0,1,2,...18]
["a",'b',...]

SyntaxError: invalid syntax (<ipython-input-14-982a9efba5a4>, line 1)

In [15]:
from sklearn.preprocessing import LabelEncoder

In [16]:
lb = LabelEncoder()  #标准化标签，将标签值统一转换成range(标签值个数-1)范围内
train['label'] = lb.fit_transform(train['class'])

In [17]:
lb.fit_transform(train['class'])

array([13,  2, 11, ..., 11,  3, 10], dtype=int64)

In [18]:
train.head()

id                                            article  \
0   0  7368 1252069 365865 755561 1044285 129532 1053...   
1   1  581131 165432 7368 957317 1197553 570900 33659...   
2   2  7368 87936 40494 490286 856005 641588 145611 1...   
3   3  299237 760651 299237 887082 159592 556634 7489...   
4   4  7368 7368 7368 865510 7368 396966 995243 37685...   

                                            word_seg  class  label  
0  816903 597526 520477 1179558 1033823 758724 63...     14     13  
1  90540 816903 441039 816903 569138 816903 10343...      3      2  
2  816903 1012629 957974 1033823 328210 947200 65...     12     11  
3  563568 1239563 680125 780219 782805 1033823 19...     13     12  
4  816903 816903 816903 139132 816903 312320 1103...     12     11

In [19]:
train['class_trans'] = lb.inverse_transform(train['label'])

F:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [20]:
train.head()

id                                            article  \
0   0  7368 1252069 365865 755561 1044285 129532 1053...   
1   1  581131 165432 7368 957317 1197553 570900 33659...   
2   2  7368 87936 40494 490286 856005 641588 145611 1...   
3   3  299237 760651 299237 887082 159592 556634 7489...   
4   4  7368 7368 7368 865510 7368 396966 995243 37685...   

                                            word_seg  class  label  \
0  816903 597526 520477 1179558 1033823 758724 63...     14     13   
1  90540 816903 441039 816903 569138 816903 10343...      3      2   
2  816903 1012629 957974 1033823 328210 947200 65...     12     11   
3  563568 1239563 680125 780219 782805 1033823 19...     13     12   
4  816903 816903 816903 139132 816903 312320 1103...     12     11   

   class_trans  
0           14  
1            3  
2           12  
3           13  
4           12

In [21]:
train[train['class'] == train['class_trans']].shape

(102277, 6)

In [22]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=4, dual=True) # c 1.0  2 3 4 5  200w特征 10w样本
clf.fit(train_term_doc,train['label'])#[0,18]
# LogisticRegression(penalty='l2',dual=False,C=1.0)   penalty-惩罚项  dual-当特征样本》样本数 一般设置为true C-类似于SVM里的软间隔

LogisticRegression(C=4, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [23]:
test_prob = clf.predict_proba(test_term_doc)  # predict_proba返回的是一个 n 行 k 列的数组， 第 i 行 第 j 列上的数值是模型预测 第 i 个预测样本为某个标签的概率，并且每一行的概率和为1。

In [ ]:
test_prob[0]# 4 +1 =5

# 结果提交

In [ ]:
test_pred = np.argmax(test_prob,axis=1) #找到最大值的索引

In [ ]:
test_pred

In [ ]:
test['class'] = lb.inverse_transform(test_pred)#[1,...19]转换成原来的标签

In [ ]:
test.head()

In [ ]:
test[["id","class"]].to_csv("submission_baseline.csv",index=False,header=True,encoding='utf-8')